In [1]:
import os
import numpy as np
import pandas as pd
from scipy.interpolate import interp1d

import xarray as xr
import datetime as dt
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

rc = {"font.family": "serif",
      "mathtext.fontset": "stix"}
plt.rcParams.update(rc)
plt.rcParams["font.serif"] = ["Times New Roman"] + plt.rcParams["font.serif"]

plt.rcParams.update({'font.size': 18, 'axes.labelsize': 18})


from pylisst import calib,driver,process,utils,__version__
# personal package
from mie_perso import psd, mie_multiprocess

opj = os.path.join

calfact = calib()

In [2]:

idir = '/DATA/projet/gernez/hablab/2023/lisst-vsf'
odir= '/DATA/projet/gernez/hablab/2023/lisst-vsf/L2'


filez_ = 'Z1071316.VSF'
info_file = opj(idir, 'LISST_VSF_Sample_Filenames_th.csv')
info = pd.read_csv(info_file)
info['Date']=pd.to_datetime(info.Date,dayfirst=True)
info

,Date,UTC time,ID,filename,Comment,median_radius
0,2022-04-20,12:57,PRY_S_C1_R1,V1101257.VSF,NaN,NaN
1,2022-04-20,13:09,PRY_S_C1_R2,V1101309.VSF,NaN,NaN
2,2022-04-20,13:16,PRY_S_C1_R3,V1101316.VSF,NaN,NaN
3,2022-04-20,14:25,PRY_S_C2_R1,V1101425.VSF,NaN,NaN
4,2022-04-20,14:31,PRY_S_C2_R2,V1101431.VSF,NaN,NaN
...,...,...,...,...,...,...
83,2023-04-20,14:17,EMI_S_C2,V1101417.VSF,NaN,NaN
84,2023-04-21,7:06,MilliQ,Z1110706.VSF,"""ZeroScattering"" background",NaN
85,2023-04-21,7:49,CHA_S_C1,V1110749.VSF,NaN,NaN
86,2023-04-21,8:34,CHA_S_C2,V1110834.VSF,NaN,NaN


In [3]:
# get info for 2023 exp.
info = info[info.Date.dt.year>2022]
info

,Date,UTC time,ID,filename,Comment,median_radius
56,2023-04-17,13:28,MilliQ,Z1071316.VSF,"""ZeroScattering"" background",NaN
57,2023-04-17,13:28,MilliQ,V1071328.VSF,"""ZeroScattering"" background",NaN
58,2023-04-17,14:17,MilliQ,V1071411.VSF,"not as background, just as regular sample mesured",NaN
59,2023-04-17,15:16,ALX_S_C1,V1071516.VSF,NaN,NaN
60,2023-04-17,16:29,BEADS1mic_S_C1,V1071629.VSF,remove samples number 60 to 70 on LISST-200X,NaN
61,2023-04-17,16:54,ALX_F_C1,V1071654.VSF,NaN,NaN
62,2023-04-18,7:52,MilliQ,V1080752.VSF,"not as background, just as regular sample mesured",NaN
63,2023-04-18,8:52,LEP_S_C1,V1080802.VSF,NaN,NaN
64,2023-04-18,9:53,LEP_S_C2,V1080953.VSF,NaN,NaN
65,2023-04-18,12:56,Saltwater_blank,V1081256.VSF,NaN,NaN


In [4]:
mie = False

idir = '/DATA/projet/gernez/hablab/2023/lisst-vsf/raw'
odir= '/DATA/projet/gernez/hablab/2023/lisst-vsf/L2'


filez_ = 'Z1071316.VSF'

info.loc[:,'c_mean']=np.nan
info['c_median']=np.nan
info['c_std']=np.nan

for irow, sample in info.iterrows():

    file_ = sample.filename

    if file_[0] == 'Z':
        continue
    name = sample.ID
    attrs = name.split('_')
    if len(attrs) == 2:
        attrs.append('0')
    if name == 'MilliQ':
        attrs = ['MilliQ','','']
    # else:
    #     continue
    print(sample.Date, name)
    # ---------------------------------
    # open and process LISST-VSF data
    # ---------------------------------
    file = os.path.join(idir, file_)
    filez = os.path.join(idir, filez_)
    scat = driver(file)
    scat.reader()
    zsc = driver(filez)
    zsc.reader()
    p = process(scat, zsc, calfact)
    p.full_process()


    c_mean,c_median,c_std = float(p.beam_c.sel(config=1).mean()),float(p.beam_c.sel(config=1).median()), float(p.beam_c.sel(config=1).std())
    info.loc[irow,['c_mean','c_median','c_std']]=c_mean,c_median,c_std

    output = xr.Dataset({
                'name': attrs[0],
                'type': attrs[1],
                'number': attrs[2],
                'beam_c':p.beam_c,
                'P11':p.P11,
                'p12':p.p12,
                'p22':p.p22})

    output.to_netcdf(opj(odir, name) + '_scat_mat.nc')

/tmp/ipykernel_1744143/2599090235.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  info.loc[:,'c_mean']=np.nan
/tmp/ipykernel_1744143/2599090235.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  info['c_median']=np.nan
/tmp/ipykernel_1744143/2599090235.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/ind

2023-04-17 00:00:00 MilliQ
2023-04-17 00:00:00 MilliQ
2023-04-17 00:00:00 ALX_S_C1
2023-04-17 00:00:00 BEADS1mic_S_C1
2023-04-17 00:00:00 ALX_F_C1
2023-04-18 00:00:00 MilliQ
2023-04-18 00:00:00 LEP_S_C1
2023-04-18 00:00:00 LEP_S_C2
2023-04-18 00:00:00 Saltwater_blank
2023-04-18 00:00:00 MIC_S_C1
2023-04-18 00:00:00 LEP_F_C2
2023-04-18 00:00:00 MIC_M_C1
2023-04-19 00:00:00 KAR_S_C1
2023-04-19 00:00:00 Saltwater_blank
2023-04-19 00:00:00 KAR_S_C2
2023-04-19 00:00:00 MilliQ
2023-04-19 00:00:00 HEC_S_C1
2023-04-19 00:00:00 HEC_S_C2
2023-04-19 00:00:00 KAR_F_C2
2023-04-19 00:00:00 HEC_F_C2
2023-04-20 00:00:00 MilliQ
2023-04-20 00:00:00 ALM_S_C1
2023-04-20 00:00:00 ALM_S_C2
2023-04-20 00:00:00 ALM_F_C2
2023-04-20 00:00:00 BEADS3mic_S_C2
2023-04-20 00:00:00 EMI_S_C1
2023-04-20 00:00:00 EMI_S_C2
2023-04-21 00:00:00 CHA_S_C1
2023-04-21 00:00:00 CHA_S_C2
2023-04-21 00:00:00 CHA_F_C2


In [5]:
info


,Date,UTC time,ID,filename,Comment,median_radius,c_mean,c_median,c_std
56,2023-04-17,13:28,MilliQ,Z1071316.VSF,"""ZeroScattering"" background",NaN,NaN,NaN,NaN
57,2023-04-17,13:28,MilliQ,V1071328.VSF,"""ZeroScattering"" background",NaN,-0.012214,0.017879,0.278506
58,2023-04-17,14:17,MilliQ,V1071411.VSF,"not as background, just as regular sample mesured",NaN,0.024949,0.025382,0.028687
59,2023-04-17,15:16,ALX_S_C1,V1071516.VSF,NaN,NaN,3.609426,3.603883,0.049770
60,2023-04-17,16:29,BEADS1mic_S_C1,V1071629.VSF,remove samples number 60 to 70 on LISST-200X,NaN,1.440710,1.438243,0.025159
61,2023-04-17,16:54,ALX_F_C1,V1071654.VSF,NaN,NaN,0.346580,0.336420,0.054920
62,2023-04-18,7:52,MilliQ,V1080752.VSF,"not as background, just as regular sample mesured",NaN,0.349070,0.221029,0.360448
63,2023-04-18,8:52,LEP_S_C1,V1080802.VSF,NaN,NaN,4.192983,4.192909,0.036360
64,2023-04-18,9:53,LEP_S_C2,V1080953.VSF,NaN,NaN,2.257456,2.258954,0.035680
65,2023-04-18,12:56,Saltwater_blank,V1081256.VSF,NaN,NaN,-0.044054,-0.045920,0.017058


In [5]:
p.beam_c

<xarray.DataArray '' (set: 75, config: 2)>
array([[0.17318269, 0.15512162],
       [0.20253662, 0.13127477],
       [0.11555763, 0.15042729],
       [0.17679932, 0.13074818],
       [0.17830047, 0.13412456],
       [0.21292254, 0.14397791],
       [0.15151707, 0.14812256],
       [0.14371405, 0.14060495],
       [0.151034  , 0.1214147 ],
       [0.16209957, 0.15303117],
       [0.16076669, 0.20009362],
       [1.51454594, 0.12815552],
       [1.33980806, 0.14889167],
       [0.16076669, 0.14397791],
       [0.16259002, 0.15748687],
       [0.18424469, 0.14320536],
       [0.13772983, 0.16577359],
       [0.16392017, 0.27492726],
       [0.12064007, 0.16687603],
       [0.13541702, 0.16729212],
...
       [0.15394198, 0.15672116],
       [0.1529701 , 0.1962692 ],
       [0.13178092, 0.14271443],
       [1.83268496, 0.17817385],
       [2.2205233 , 0.15933954],
       [0.14379628, 0.17667785],
       [0.15760295, 0.16348935],
       [0.15577335, 0.15857382],
       [1.13117643, 0.13855167],
       [0.15626123, 0.16086839],
       [0.1326808 , 0.1518022 ],
       [0.14978959, 0.18948033],
       [0.15577335, 0.1759233 ],
       [0.14930591, 0.1751679 ],
       [1.17697711, 0.14918983],
       [0.15210883, 0.14918983],
       [0.14515352, 0.15257135],
       [0.1617476 , 0.12502215],
       [0.15809179, 0.1468746 ],
       [0.16554628, 0.15627146]])
Coordinates:
  * set      (set) int64 0 1 2 3 4 5 6 7 8 9 ... 65 66 67 68 69 70 71 72 73 74
  * config   (config) int64 0 1
    pmt      (set) int64 435 510 510 510 510 510 510 ... 510 510 510 510 510 510